<h1 align='center'> CSC4120 Programming Assignment 3  </h1>

## Submission Requirements

   The submission <font color = #FF0000>deadline is March 17 (Sun.), 2024, 11:59 pm</font>. Solutions submitted after the deadline will be graded as 0 points. Please submit a **ipynb** file using given template and clearly state your group members' student IDs. Otherwise, your points will be deducted.

## Studen IDs

120090244

122090031

121090271


## Remote Error Correction

Prof. Costas was isolated in Xi'an. He was planning to work on the homework during the isolation. Unfortunately, his computer broke and corrupted the latest copy of the homework file. A few lines of the file were affected. Prof. Costas could just ask Mingjie, who was in Shenzhen, to send him the latest copy of the file. But the hotel he stayed has very slow and pricey Internet access.

Help Prof. Costas recover the file and prepare homework on time! Design an interactive protocol for detecting and correcting corrupted lines that uses **little communication**.

In this question, you have to answer two short-answer questions and implement the following three functions in file_compare.ipynb.
(Feel free to add additional methods)

* def hash_function(s)
* def compute_node_value(table, i, j)
* def binary_check_hash(left, right)

First, Prof. Costas and Mingjie need a good hash function that can compute a hash value for any contiguous subset of lines of a file.

1. Implement a **division hash** in the function `hash_function`.

You can use bulit-in function `hash()` (or any other function) to calculate the hash code of a string, but you should make **your** `hash_function` a division hash. 

2. Implement the function `compute_node_value`.

It computes all useful hash values for binary comparison (to be explained later), and store them in corresponding table (`HashCorruptedFile[][]` and `HashOriginalFile[][]`). It only calculates `HashCorruptedFile[0, n]`, `HashCorruptedFile[0, n/2]`, `HashCorruptedFile[0, n/4]`, `HashCorruptedFile[n/2, n]`, `HashCorruptedFile[n/4, n/2]` ... etc.

Hints:
- `wordsCorrupted[i]` := data of line i in the corrupted file
- `HashCorruptedFile[i][i]` := Hash value of "line i in the corrupted File"
- `HashCorruptedFile[i][j]` := Hash value of "lines i to j in the corrupted File"

For example, if the corrupted file (4 lines) is as follows:

```raw
    aaaa\n
    bbbb\n
    cccc\n
    dddd
```

then,
- `wordsCorrupted[0]` = "aaaa"
- `HashCorruptedFile[0][0]` = hash function("aaaa")
- `HashCorruptedFile[1][1]` = hash function("bbbb")
- `HashCorruptedFile[0][2]` = hash function(temp). temp is something related to "aaaa",
"bbbb" and "cccc", or related to their hash value. It depends on your own design.

----


Now Prof. Costas and Mingjie have your hash function. They want to find the corrupted lines. Mingjie came up with a naive algorithm to compare both files line by line (and he wrote the codes in `naive_check_hash`). They could send and compare the hash value of each line. But Prof. Costas is unsatisfied with the communication cost $ \Theta(n) $. Help them design an efficient algorithm! 

3. Suppose the two copies of the file differ in some number of consecutive lines.
Let $ n $ be the total number of lines in the file. How can Prof. Costas and Mingjie use
binary search to find the start and end of the corruption by exchanging only $ O(\log n) $ hash
values? Describe your idea below.

\###

Pseudo code is below:


        
           


\###


In [ ]:
def binary_check_hash(left, right):
    if left == right:
        return -1, largest_integer
    mid = (left + right) // 2
    if HashCorruptedFile[left][mid] != HashOriginalFile[left][mid]:
        min = binary_check_hash(left, mid)
        max = binary_check_hash(mid + 1, right)
        return min, max
    else:
        min1, max1 = binary_check_hash(left, mid)
        min2, max2 = binary_check_hash(mid + 1, right)
        return min(min1, max1), max(min1, max1)

def find_min_corrupted_line(left,right):
    if left == right:
        return left
    mid = (left + right) // 2
    if HashCorruptedFile[left][mid] != HashOriginalFile[left][mid]:
        return find_min_corrupted_line(left, mid)
    else:
        return find_min_corrupted_line(mid + 1, right)
    

def find_max_corrupted_line(left,right):
    if left == right:
        return left
    mid = (left + right) // 2
    if HashCorruptedFile[mid][right] != HashOriginalFile[mid][right]:
        return find_max_corrupted_line(mid, right)
    else:
        return find_max_corrupted_line(left, mid-1)

4. Let $ W $ ($ W \ll n $) be the number of corrupted lines (not necessarily consecutive). What can
they do to find all corrupted lines by exchanging only $ O(W · \log n) $ hash values? Describe your idea below.

\###

By applying the binary check hash for $ W $ times, they can find all corrupted lines by exchanging only $ O(W · \log n) $ hash values.

\###


5. Implement `binary_check_hash`.

This function simulates the interactive protocol (and is the implementation of your idea in 4). It should call the function `compare` to compare two hash values, print `"[binary_check_hash] found damaged content at line xx"` as `naive_check_hash` does whenever a corrupted line is found, and call the function `send_words` to retrieve the original line from the server.

In [4]:

# Hash Values for words in New File
# HashCorruptedFile[i,j] = Hash value of lines i to j in Corrupted File
HashCorruptedFile = []

# Hash Values for words in Old File
# HashOriginalFile[i,j] = Hash value of lines i to j in Original File
HashOriginalFile = []

# Words in the Corrupted and Original File
wordsCorrupted = []
wordsOriginal = []

# n : number of lines in each file
n = 1 # default value

# variable to measure network transmission cost
cost = 0

# penalty value for transmitting the whole line
penalty = 4


# feel free to add additional functions

def hash_function(s):
    '''
    compute and return the hash value of the string s.

    You can use built-in hash() (or any other method) to calculate a hash code. But you should finally return a division hash code (something % something). 
    '''
    #######################
    #                     #
    #      Question 1     #
    #     TO IMPLEMENT    #
    #                     #
    #######################
    return hash(s) % 10007


def compute_node_values():
    ''' 
    in the first part of compute_hash_tree(), we computed HashOriginalFile[0][0], HashOriginalFile[1][1], HashOriginalFile[2][2].... etc
    
    Now, here in compute_node_values(), you have to calculate HashOriginalFile[i][j] where i < j
    So, HashOriginalFile[2][4] = hash_function("line 2-4 of corrupted file"). It depends on your own design. 
    
    For example, HashOriginalFile[2][4] can be = hash_function(hash_function(line 2) + 2*hash_function(line 3) + 3*hash_function(line 4)).... but it is good? Perhaps not. So, write your own hash function here! 
    '''
    compute_node_value(HashCorruptedFile, 0, n - 1)
    compute_node_value(HashOriginalFile, 0, n - 1)

def compute_node_value(table, i, j):
    '''
    it calculates table[i][j], where table could be HashCorruptedFile or HashOriginalFile.

    Remember you are doing a binary search here. It's not necessary to try out all combination of i, j. You only need to consider (0, n), (0, n/2), (n/2, n), ... 
    '''
    #######################
    #                     #
    #      Question 2     #
    #     TO IMPLEMENT    #
    #                     #
    #######################
    if i == j:
        return
    mid = (i + j) // 2
    compute_node_value(table, i, mid)
    compute_node_value(table, mid + 1, j)
    table[i][j] = hash_function(table[i][mid] + table[mid + 1][j])



def binary_check_hash(left, right):
    ''' 
    performs binary search over the hash tree.

    Search through the whole tree. 
    The output should be similar to naive_check_hash().
    '''
    global cost, penalty
    #######################
    #                     #
    #      Question 5     #
    #     TO IMPLEMENT    #
    #                     #
    #######################
    if left == right:
        if not compare(HashCorruptedFile[left][left], HashOriginalFile[left][left]):
            print("[binary_check_hash] found damaged content at line", left)
            send_word()
    else:
        mid = (left + right) // 2
        if not compare(HashCorruptedFile[left][mid], HashOriginalFile[left][mid]):
            binary_check_hash(left, mid)
        if not compare(HashCorruptedFile[mid + 1][right], HashOriginalFile[mid + 1][right]):
            binary_check_hash(mid + 1, right)


# read fileCorrupted and fileOriginal, save to wordsCorrupted and wordsOriginal
def read_files(fileCorrupted, fileOriginal):
    print("reading files:", fileCorrupted, fileOriginal)
    with open(fileCorrupted, 'r') as fCorrupted:
        for line in fCorrupted:
            wordsCorrupted.append(line.rstrip())
    with open(fileOriginal, 'r') as fOriginal:
        for line in fOriginal:
            wordsOriginal.append(line.rstrip())


# Compute the hash values of the leaf nodes in the Hash Tree
def compute_hash_tree():
    for i in range(n):
        HashCorruptedFile.append([])
        HashOriginalFile.append([])
        for _ in range(n):
            HashCorruptedFile[i].append(None)
            HashOriginalFile[i].append(None)

    for i in range(n):
        HashCorruptedFile[i][i] = hash_function(wordsCorrupted[i])
        HashOriginalFile[i][i] = hash_function(wordsOriginal[i])

    compute_node_values()


# compares the two hash values. total cost increase by 1
def compare(oldhash, newhash):
    global cost
    cost += 1
    return oldhash == newhash


# adds the penalty of sending the whole line over network transmission
def send_word():
    global cost, penalty
    cost += penalty

# naive hash check method, goes over all the lines and compares them.
def naive_check_hash(left, right):
    global cost, penalty
    for i in range(left, right + 1):
        if not compare(HashCorruptedFile[i][i], HashOriginalFile[i][i]):
            print("[naive_check_hash] found damaged content at line", i)
            send_word()


# performs the naive_check_hash algorithm and binary_check_hash algorithm.
# compares filename1 and filename2 with n lines each
# print out the cost
def file_transfer(filename1, filename2, n1):
    global cost, n
    n = n1

    print("1. starting computation of hash tree...")
    read_files(filename1, filename2)
    compute_hash_tree()
    print("finished computation of hash tree\n")

    print("2. starting computation of naive_check_hash()...")
    cost = 0
    naive_check_hash(0, n - 1)
    print("cost of navie_check_hash() =", cost, "\n")

    print("3. starting computation of binary_check_hash()...")
    cost = 0
    binary_check_hash(0, n - 1)
    print("cost of binary_check_hash() =", cost, "\n")


if __name__ == "__main__":
    file_transfer("file_corrupted1.txt", "file_original1.txt", 1024)
    file_transfer("file_corrupted2.txt", "file_original2.txt", 4096)
    file_transfer("file_corrupted3.txt", "file_original3.txt", 16384)

1. starting computation of hash tree...
reading files: file_corrupted1.txt file_original1.txt
finished computation of hash tree

2. starting computation of naive_check_hash()...
[naive_check_hash] found damaged content at line 255
[naive_check_hash] found damaged content at line 511
[naive_check_hash] found damaged content at line 767
[naive_check_hash] found damaged content at line 1023
cost of navie_check_hash() = 1040 

3. starting computation of binary_check_hash()...
[binary_check_hash] found damaged content at line 255
[binary_check_hash] found damaged content at line 511
[binary_check_hash] found damaged content at line 767
[binary_check_hash] found damaged content at line 1023
cost of binary_check_hash() = 90 

1. starting computation of hash tree...
reading files: file_corrupted2.txt file_original2.txt
finished computation of hash tree

2. starting computation of naive_check_hash()...
[naive_check_hash] found damaged content at line 255
[naive_check_hash] found damaged content